In [1]:
x = 5
y = "hello"
z = True

In [2]:
x.__hash__()

5

In [3]:
y.__hash__()

-1048367596172050431

In [4]:
z.__hash__()

1

In [5]:
x.__str__()

'5'

In [6]:
from langchain.chat_models import init_chat_model
def get_llm(name, provider):
    return init_chat_model(model=name, model_provider=provider)

def ask_llm(llm, question):
    response = llm.invoke(question)
    response.pretty_print()

In [7]:
model_name = "gemini-2.5-flash"
model_provider = "google_vertexai"

gemini_llm = get_llm(model_name, model_provider)

In [8]:
type(gemini_llm)

langchain_google_vertexai.chat_models.ChatVertexAI

In [9]:
response = gemini_llm.invoke("What is capital of france?")

In [10]:
type(response)

langchain_core.messages.ai.AIMessage

In [11]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [12]:
# few shot prompt

messages = [
    SystemMessage(content="You are an expert in mathematics"),
    HumanMessage(content="I have 2 pens now i bought 2 more. What is total number of pens?"),
    AIMessage(content="This refers to addition. 2 + 2 = 4 "),
    HumanMessage(content="I have 10 pens now i lost 2. What is total number of pens?"),
    AIMessage(content="This refers to subtraction. 10 - 2 = 8 "),
    HumanMessage(content="I have 10 bikes i sold one, How many bikes do i have left ?")

]

In [13]:
response = gemini_llm.invoke(messages)

In [14]:
response

AIMessage(content='This is another subtraction problem.\n\n10 - 1 = 9\n\nYou have **9 bikes left**.', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 91, 'candidates_token_count': 23, 'total_token_count': 155, 'prompt_tokens_details': [{'modality': 1, 'token_count': 91}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 23}], 'thoughts_token_count': 41, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.39093059042225714, 'model_name': 'gemini-2.5-flash'}, id='run--8186eca2-abdb-4c87-b61c-f21ef603553f-0', usage_metadata={'input_tokens': 91, 'output_tokens': 23, 'total_tokens': 155, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 41}})

In [15]:
response.pretty_print()

================================== Ai Message ==================================

This is another subtraction problem.

10 - 1 = 9

You have **9 bikes left**.


In [16]:
from langchain_core.tools import tool

@tool(parse_docstring=True, return_direct=True )
def add(a: int, b: int) -> int:
    """Adds a,  b

    Args:
        a (int): a value
        b (int): b value

    Returns:
        int: a + b
    """
    return a + b

@tool(parse_docstring=True, return_direct=True )
def sub(a: int, b: int) -> int:
    """Performs subtraction of a, b

    Args:
        a (int): a value
        b (int): b value

    Returns:
        int: a - b
    """
    return a - b

In [17]:
# model
gemini_llm_with_tools = gemini_llm.bind_tools([add, sub])

In [18]:
message = HumanMessage(content="I have 10 bikes i sold one, How many bikes do i have left ?")

In [20]:
response = gemini_llm_with_tools.invoke([message])

In [22]:
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  sub (13ead190-2487-4a7a-8eb1-e7ec5849dae4)
 Call ID: 13ead190-2487-4a7a-8eb1-e7ec5849dae4
  Args:
    b: 1.0
    a: 10.0


In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
#from langgraph.prebuilt import create_react_agent


In [24]:
server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["mcp_math.py"],
)

In [25]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)
        gemini_llm_with_mcp_tools = gemini_llm.bind_tools(tools)
        response = await gemini_llm_with_mcp_tools.ainvoke([HumanMessage(content="I have 10 bikes i sold one, How many bikes do i have left ?")])

        # Create and run the agent
        # agent = create_react_agent("openai:gpt-4.1", tools)
        # agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

UnsupportedOperation: fileno